# Heave → ARIMA/ETS → FFT (fase zero) → Zero-Downcrossing

Notebook **análogo** ao pipeline com Prophet, agora com **ARIMA/SARIMAX** e **ETS**.

## Objetivo
- Prever **heave** com **ARIMA/SARIMAX** e **ETS**;
- Aplicar **FFT** para **regularização espectral** (passa-faixa, fase zero);
- Derivar **N, Hs, Hmean, Hmax, Tz** via **zero-downcrossing**;
- Avaliar vs. **Persistência** em *rolling origin* (+30, +60, +120 passos).

## Dependências
`pandas`, `numpy`, `matplotlib`, `scipy`, `statsmodels`.

```bash
pip install pandas numpy matplotlib scipy statsmodels
```

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import welch
from numpy.fft import rfft, irfft, rfftfreq
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from typing import Tuple, Dict
plt.rcParams['figure.figsize']=(10,4)
pd.options.display.float_format = '{:.6f}'.format

## Configuração & Leitura de Dados
Esperado um CSV com `timestamp` e `heave`.

In [3]:
# === CONFIG ===
CSV_PATH = './dados/out/heave_6613_concat.csv'
TIME_COL = 'time'
VAL_COL  = 'Upward'
FREQ     = None  # ex.: 'T' se for minutely; None para inferir
HORIZONS = [30,60,120]
F_MIN_HZ = 1/1000
F_MAX_HZ = 1/4
WELCH_WINDOW='hann'; WELCH_SEG=256; WELCH_OVERLAP=0.5
HMIN=0.0

# df = pd.read_csv(CSV_PATH)
# df[TIME_COL] = pd.to_datetime(df[TIME_COL])
# df = df.sort_values(TIME_COL).set_index(TIME_COL)
# df = df[[VAL_COL]].dropna()
# df = df.asfreq(FREQ or pd.infer_freq(df.index))
# fs = 1.0/(df.index.to_series().diff().median().total_seconds())
# print('fs (Hz):', fs)
# df.head()

In [4]:
# # === CONFIG ===
# CSV_PATH = './dados/out/heave_6613_concat.csv'
# TIME_COL = 'time'
# VAL_COL  = 'Upward'
# FREQ     = None  # deixe None para auto; pode forçar 'S','T','500L' etc.

df = pd.read_csv(CSV_PATH)
df[TIME_COL] = pd.to_datetime(df[TIME_COL], utc=False, errors='coerce')
df = df.dropna(subset=[TIME_COL, VAL_COL]).sort_values(TIME_COL).set_index(TIME_COL)
df = df[[VAL_COL]].astype(float)

# 1) tratar duplicatas (escolha UMA das políticas abaixo):

# (a) manter a primeira ocorrência
# df = df[~df.index.duplicated(keep='first')]

# (b) ou agregar duplicatas por média (recomendado para sensores)
df = df.groupby(level=0).mean()

# 2) inferir passo temporal robusto (mediana dos deltas)
dt = df.index.to_series().diff().dropna()
# se tiver timezone, remova ou padronize antes
step_seconds = dt.dt.total_seconds().median()

# opcional: se você já souber a frequência, force aqui:
if FREQ is not None:
    rule = FREQ  # ex.: 'S' segundo, 'T' minuto, '500L' 500 ms
else:
    # cria uma regra a partir do passo mediano
    # arredonda para as unidades usuais (s, ms) se fizer sentido
    from pandas.tseries.frequencies import to_offset
    rule = to_offset(pd.Timedelta(seconds=step_seconds)).freqstr

# 3) reamostrar para uma grade regular (evita problemas do asfreq com jitter)
# use .mean() (ou .nearest()) e depois complete lacunas com interpolação temporal
df = df.resample(rule).mean()

# preencher pequenas lacunas de forma segura (ajuste o limite conforme seu caso)
df[VAL_COL] = df[VAL_COL].interpolate(method='time', limit=5)  # até 5 intervalos seguidos

# 4) frequência de amostragem (Hz) a partir da grade regular
if len(df.index) >= 2:
    fs = 1.0 / (df.index[1] - df.index[0]).total_seconds()
else:
    raise ValueError("Série muito curta após o pré-processamento.")

print('Regra de reamostragem:', rule, '| fs (Hz):', fs)
df.head()


Regra de reamostragem: 160ms | fs (Hz): 6.25


,Upward
time,
2025-07-14 00:00:00.000,0.030000
2025-07-14 00:00:00.160,-0.060000
2025-07-14 00:00:00.320,-0.120000
2025-07-14 00:00:00.480,-0.140000
2025-07-14 00:00:00.640,-0.120000


## Utilitários (detrend, Welch/FFT, filtro fase-zero, zero-downcrossing)

In [5]:
def detrend_mean(x):
    x = np.asarray(x)
    return x - np.nanmean(x)

def psd_welch(x, fs, nperseg=256, noverlap=0.5, window='hann'):
    n_ov = int(nperseg*noverlap)
    f,Pxx = welch(x, fs=fs, window=window, nperseg=nperseg, noverlap=n_ov, detrend=False)
    return f,Pxx

def bandpass_fft_phase_zero(x, fs, fmin, fmax):
    x = detrend_mean(x)
    N = len(x)
    X = rfft(x)
    freqs = rfftfreq(N, d=1/fs)
    mask = (freqs>=fmin)&(freqs<=fmax)
    Xf = X*mask
    return irfft(Xf, n=N)

def zero_downcrossing(h, fs, hmin=0.0):
    h = np.asarray(h)
    N = len(h)
    idx = np.where((h[:-1]>0)&(h[1:]<=0))[0]
    if len(idx)<2:
        return dict(N=0,Hmean=np.nan,Hmax=np.nan,Hs=np.nan,Tz=np.nan)
    t = np.arange(N)/fs
    t_cross = t[idx] + (h[idx]/(h[idx]-h[idx+1]))*(1/fs)
    Hs_list=[]; T_list=[]; H_list=[]
    for k in range(len(t_cross)-1):
        t0,t1 = t_cross[k], t_cross[k+1]
        i0,i1 = int(np.floor(t0*fs)), int(np.ceil(t1*fs))
        if i1<=i0+1: continue
        seg = h[i0:i1+1]
        crest = np.max(seg); trough = np.min(seg)
        H = crest - trough
        if H < hmin: continue
        H_list.append(H); T_list.append(t1-t0)
    if not H_list:
        return dict(N=0,Hmean=np.nan,Hmax=np.nan,Hs=np.nan,Tz=np.nan)
    H_arr = np.array(H_list); T_arr=np.array(T_list)
    Hmean = H_arr.mean(); Hmax=H_arr.max()
    top_n = max(1,int(np.ceil(len(H_arr)/3)))
    Hs = np.sort(H_arr)[::-1][:top_n].mean()
    Tz = T_arr.mean()
    return dict(N=len(H_arr), Hmean=Hmean, Hmax=Hmax, Hs=Hs, Tz=Tz)

def metrics(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred=np.asarray(y_pred)
    e = y_pred - y_true
    rmse = float(np.sqrt(np.mean(e**2)))
    mae  = float(np.mean(np.abs(e)))
    smape = float(np.mean(2*np.abs(e)/np.where((np.abs(y_true)+np.abs(y_pred))==0,1,(np.abs(y_true)+np.abs(y_pred)))))
    return dict(RMSE=rmse, MAE=mae, sMAPE=smape)

## Modelos: Persistência, ARIMA/SARIMAX, ETS

In [6]:
def forecast_persistence(series, steps):
    return np.repeat(series.iloc[-1], steps)

def forecast_arima(series, steps, order=(1,0,0), seasonal_order=(0,0,0,0)):
    model = SARIMAX(series, order=order, seasonal_order=seasonal_order,
                    enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)
    return res.forecast(steps=steps).values

def forecast_ets(series, steps, trend=None, seasonal=None, seasonal_periods=None):
    model = ExponentialSmoothing(series, trend=trend, seasonal=seasonal,
                                 seasonal_periods=seasonal_periods, initialization_method='estimated')
    res = model.fit(optimized=True)
    return res.forecast(steps).values

## Rolling Origin (comparando modelos e derivando parâmetros)

In [7]:
from collections import defaultdict

def rolling_origin_eval(df, val_col, horizons, fs,
                        train_min_points=500,
                        arima_order=(1,0,0), arima_seasonal=(0,0,0,0),
                        ets_trend=None, ets_seasonal=None, ets_sp=None,
                        fmin=1/1000, fmax=1/4, hmin=0.0):
    y = df[val_col].astype(float)
    res_rows=[]
    for cut in range(train_min_points, len(y)-max(horizons)-1):
        y_train = y.iloc[:cut]
        for H in horizons:
            y_true = y.iloc[cut:cut+H].values
            # modelos
            y_pers = forecast_persistence(y_train, H)
            try:
                y_arima = forecast_arima(y_train, H, order=arima_order, seasonal_order=arima_seasonal)
            except Exception:
                y_arima = np.full(H, np.nan)
            try:
                y_ets = forecast_ets(y_train, H, trend=ets_trend, seasonal=ets_seasonal, seasonal_periods=ets_sp)
            except Exception:
                y_ets = np.full(H, np.nan)

            # filtragem FFT fase-zero
            y_true_f = bandpass_fft_phase_zero(y_true, fs, fmin, fmax)
            y_pers_f = bandpass_fft_phase_zero(y_pers, fs, fmin, fmax)
            y_arima_f = bandpass_fft_phase_zero(y_arima, fs, fmin, fmax) if np.isfinite(y_arima).all() else y_arima
            y_ets_f   = bandpass_fft_phase_zero(y_ets,   fs, fmin, fmax) if np.isfinite(y_ets).all()   else y_ets

            # métricas de heave
            m_p = metrics(y_true, y_pers)
            m_a = metrics(y_true, y_arima) if np.isfinite(y_arima).all() else {k:np.nan for k in ['RMSE','MAE','sMAPE']}
            m_e = metrics(y_true, y_ets)   if np.isfinite(y_ets).all()   else {k:np.nan for k in ['RMSE','MAE','sMAPE']}

            # parâmetros por zero-downcrossing
            w_true = zero_downcrossing(y_true_f, fs, hmin)
            w_pers = zero_downcrossing(y_pers_f, fs, hmin)
            w_arim = zero_downcrossing(y_arima_f, fs, hmin) if np.isfinite(y_arima_f).all() else {k:np.nan for k in ['N','Hmean','Hmax','Hs','Tz']}
            w_ets  = zero_downcrossing(y_ets_f,  fs, hmin) if np.isfinite(y_ets_f).all()  else {k:np.nan for k in ['N','Hmean','Hmax','Hs','Tz']}

            for name, m, w in [('Persistence', m_p, w_pers), ('ARIMA', m_a, w_arim), ('ETS', m_e, w_ets)]:
                res_rows.append({
                    'cut_idx':cut, 'horizon':H, 'model':name,
                    'RMSE':m['RMSE'], 'MAE':m['MAE'], 'sMAPE':m['sMAPE'],
                    'N_true':w_true.get('N'), 'Hmean_true':w_true.get('Hmean'), 'Hmax_true':w_true.get('Hmax'), 'Hs_true':w_true.get('Hs'), 'Tz_true':w_true.get('Tz'),
                    'N_pred':w.get('N'), 'Hmean_pred':w.get('Hmean'), 'Hmax_pred':w.get('Hmax'), 'Hs_pred':w.get('Hs'), 'Tz_pred':w.get('Tz'),
                })
    return pd.DataFrame(res_rows)

### Rodar (ajuste `train_min_points` conforme o tamanho da sua série)

In [ ]:
df_res = rolling_origin_eval(df, VAL_COL, HORIZONS, fs,
    train_min_points=500,
    arima_order=(1,0,0), arima_seasonal=(0,0,0,0),
    ets_trend=None, ets_seasonal=None, ets_sp=None,
    fmin=F_MIN_HZ, fmax=F_MAX_HZ, hmin=HMIN)
df_res.head()

/home/alex/anaconda3/envs/py310/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/alex/anaconda3/envs/py310/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/alex/anaconda3/envs/py310/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/alex/anaconda3/envs/py310/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/alex/anaconda3/envs/py

## Sumário de métricas (heave e parâmetros)

In [ ]:
def relerr(true, pred):
    return 100.0*(np.asarray(pred)-np.asarray(true))/np.where(np.asarray(true)==0, np.nan, np.asarray(true))

def summarize(df_res):
    out=[]
    for (H, model), g in df_res.groupby(['horizon','model']):
        hs_rel = relerr(g['Hs_true'], g['Hs_pred'])
        tz_rel = relerr(g['Tz_true'], g['Tz_pred'])
        out.append({'horizon':H,'model':model,
                    'RMSE':g['RMSE'].mean(), 'MAE':g['MAE'].mean(), 'sMAPE':g['sMAPE'].mean(),
                    'Hs_relerr_pct_mean':np.nanmean(hs_rel), 'Tz_relerr_pct_mean':np.nanmean(tz_rel)})
    return pd.DataFrame(out)

summary = summarize(df_res)
summary.sort_values(['horizon','model'])

## PSD (Welch) para ajuste de banda

In [ ]:
x = df[VAL_COL].astype(float).values
x_d = x - np.nanmean(x)
f_psd, Pxx = psd_welch(x_d, fs, nperseg=WELCH_SEG, noverlap=WELCH_OVERLAP, window=WELCH_WINDOW)
plt.plot(f_psd, Pxx)
plt.xlabel('Frequência (Hz)'); plt.ylabel('PSD'); plt.title('Welch PSD - Heave (detrended)'); plt.show()

## Exportar resultados (CSV)

In [ ]:
from pathlib import Path
outdir = Path('./outputs_arima_ets')
outdir.mkdir(parents=True, exist_ok=True)
df_res.to_csv(outdir/'rolling_results.csv', index=False)
summary.to_csv(outdir/'summary_metrics.csv', index=False)
print('Arquivos salvos em', outdir.resolve())

## Referências (ABNT)
- BOX, G. E. P.; JENKINS, G. M.; REINSEL, G. C.; LJUNG, G. M. **Time Series Analysis: Forecasting and Control**. 5. ed. Wiley, 2015.
- HYNDMAN, R. J.; ATHANASOPOULOS, G. **Forecasting: Principles and Practice**. 3. ed. OTexts, 2021.
- HOLTHUIJSEN, L. H. **Waves in Oceanic and Coastal Waters**. Cambridge University Press, 2007.
- GODA, Y. **Random Seas and Design of Maritime Structures**. 3. ed. World Scientific, 2010.
- OPPENHEIM, A. V.; SCHAFER, R. W. **Discrete-Time Signal Processing**. 3. ed. Prentice Hall, 2010.
- WELCH, P. D. (1967). *The use of fast Fourier transform for the estimation of power spectra*. IEEE Trans. Audio and Electroacoustics, 15(2), 70–73.